In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from validate import validate

In [ ]:
data_dir = './data'
locations = ['muOsna','wernig','braunl','redlen']
station = locations[3]

In [ ]:
id_predictors   = np.loadtxt(data_dir+'/id_predictors/'+station+'.txt', dtype=np.int32, delimiter=',') - 1 #Matlab index start in 1

trn_t = np.load(data_dir + '/' + station + '/trn_t.npy')
trn_x = np.load(data_dir + '/' + station + '/trn_x.npy')
trn_x = trn_x[:, id_predictors]
trn_y = np.load(data_dir + '/' + station + '/trn_y.npy')
print(trn_x.shape, '--->', trn_y.shape)

trn_t = trn_y[trn_y > 0]
trn_x = trn_x[trn_y > 0]
trn_y = trn_y[trn_y > 0]
print(trn_x.shape, '--->', trn_y.shape)

In [ ]:
depth = [1,2,3,4]
units = [4,8,16,32]
runs  = 5

In [ ]:
results = np.empty([len(depth), len(units), runs])

In [ ]:
batch_size = 16
epochs = 256

for idx_d, d in enumerate(depth):
    for idx_u, u in enumerate(units):        
        for r in range(runs):

            optimizer = tf.optimizers.Adam(learning_rate=0.0001)
            loss = tf.keras.losses.MeanSquaredLogarithmicError()
            early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5)
            activation = 'relu'
            model = tf.keras.Sequential()

            model.add(tf.keras.layers.InputLayer(input_shape=(trn_x.shape[1])))
            for i in range(d):
                model.add(tf.keras.layers.Dense(u, activation=activation))

            model.add(tf.keras.layers.Dense(1))
            model.compile(optimizer, loss)

            hist = model.fit(trn_x, trn_y, batch_size, epochs, validation_split=0.2, shuffle=True, verbose=0, callbacks=[early_stop])
            val_loss = np.min(hist.history['val_loss'])
            fig, ax = plt.subplots()
            ax.plot(hist.history['loss'], label='loss')
            ax.plot(hist.history['val_loss'], label='val_loss')
            fig.legend()
            fig.savefig('search_arch/' + station + '/depth' + str(d) + 'units' + str(u) + 'run' +str(r)+ '.png')
            
            results[idx_d, idx_u, r] = val_loss
            
            print(d, u, r)
            print(val_loss)
            
np.save('search_arch/' + station + '/' + station + '_searh_arch.npy', results)